In [146]:
import pandas as pd
import os,sys,codecs,csv,pickle,string
from pprint import pprint
import numpy as np
from stop_words import get_stop_words
import multiprocessing



In [140]:
# all files
#csv_files = [f for f in os.listdir() if f.endswith('.csv') ]
#csv_files = [(d,s) for d in range(4000) for s in csv_files if s.endswith('FY'+str(d)+'.csv') ]

# only abstract files
csv_files = [f for f in os.listdir() if (f.endswith('.csv') and f.find('_PRJABS_')>-1) ]
csv_files = [(d,s) for d in range(4000) for s in csv_files if s.endswith('FY'+str(d)+'.csv') ]
pprint(csv_files)


[(2006, 'RePORTER_PRJABS_C_FY2006.csv'),
 (2007, 'RePORTER_PRJABS_C_FY2007.csv'),
 (2008, 'RePORTER_PRJABS_C_FY2008.csv'),
 (2009, 'RePORTER_PRJABS_C_FY2009.csv'),
 (2010, 'RePORTER_PRJABS_C_FY2010.csv'),
 (2011, 'RePORTER_PRJABS_C_FY2011.csv'),
 (2012, 'RePORTER_PRJABS_C_FY2012.csv'),
 (2013, 'RePORTER_PRJABS_C_FY2013.csv'),
 (2014, 'RePORTER_PRJABS_C_FY2014.csv'),
 (2015, 'RePORTER_PRJABS_C_FY2015.csv'),
 (2016, 'RePORTER_PRJABS_C_FY2016.csv'),
 (2017, 'RePORTER_PRJABS_C_FY2017.csv'),
 (2018, 'RePORTER_PRJABS_C_FY2018.csv'),
 (2019, 'RePORTER_PRJABS_C_FY2019.csv')]


In [132]:

# this will produce two lists, one entry per grant
#   first list has grant year.   second list has grant abstract text
years = list()
abstracts = list()
table = str.maketrans(dict.fromkeys(string.punctuation))
for I in range(0,len(csv_files),1): # len(csv_files)
    print(I,csv_files[I][0] , csv_files[I][1])
    with codecs.open(csv_files[I][1] , 'r' , encoding='utf-8', errors='ignore') as csvfile:
        csvr = csv.reader(csvfile,delimiter=',')
        for app_id, app_txt in csvr:
            years.append(csv_files[I][0])
            app_txt = [s.translate(table) for s in app_txt.lower().split(' ')]
            filtered_words = [word for word in app_txt  if \
                              ( (word not in get_stop_words('english')) and (len(word)>4) )]
            abstracts.append( set(filtered_words) )

years = np.array(years)
abstracts = np.array(abstracts)

0 2010 RePORTER_PRJABS_C_FY2010.csv
1 2011 RePORTER_PRJABS_C_FY2011.csv
2 2012 RePORTER_PRJABS_C_FY2012.csv
3 2013 RePORTER_PRJABS_C_FY2013.csv
4 2014 RePORTER_PRJABS_C_FY2014.csv
5 2015 RePORTER_PRJABS_C_FY2015.csv
6 2016 RePORTER_PRJABS_C_FY2016.csv
7 2017 RePORTER_PRJABS_C_FY2017.csv
8 2018 RePORTER_PRJABS_C_FY2018.csv
9 2019 RePORTER_PRJABS_C_FY2019.csv


In [ ]:
print(abstracts[2])
print('---')
with open('abstracts.pickle', 'wb') as f:
    pickle.dump(abstracts, f, pickle.HIGHEST_PROTOCOL)
with open('years.pickle', 'wb') as f:
    pickle.dump(years, f, pickle.HIGHEST_PROTOCOL)

with open('abstracts.pickle', 'rb') as f:
    abstracts = pickle.load(f) 

with open('years.pickle', 'rb') as f:
    years = pickle.load(f)     

print(abstracts[2])

In [135]:
def CountGrantsInOneYearWithWord( grants_abs_list , word):
    binary_vect = [word in words for words in grants_abs_list]
    return( sum(binary_vect), sum(binary_vect)/len(binary_vect)*100, len(binary_vect) )

# load an abstracts CSV file and return a set of unique, no-stop & no-punctuation words
def LoadWordsFromAbstractFile(abstract_csv_file):
    table = str.maketrans(dict.fromkeys(string.punctuation))
    with codecs.open( abstract_csv_file , 'r' , encoding='utf-8', errors='ignore') as csvfile:
        csvr = csv.reader(csvfile,delimiter=',')
        for app_id, app_txt in csvr:
            app_txt = [s.translate(table) for s in app_txt.lower().split(' ')]
            filtered_words = [word for word in app_txt  if \
                              ( (word not in get_stop_words('english')) and (len(word)>4) )]
    return( set(filtered_words) )

In [118]:
s,m,l = CountGrantsInOneYearWithWord( abstracts[years==2010] , 'yeast')
print(s,m,l)

1773 2.0356380169464283 87098


In [154]:
# https://www.quantstart.com/articles/Parallelising-Python-with-Threading-and-Multiprocessing/
jobs = []
for I in range(len(csv_files)):
    print(I,csv_files[I][1])
    out_list = list()
    process = multiprocessing.Process(target=LoadWordsFromAbstractFile,args=csv_files[I][1])
    jobs.append(process)

# Start the processes (i.e. calculate the random number lists)      
for j in jobs:
    j.start()

# Ensure all of the processes have finished
for j in jobs:
    j.join()

#print("List processing complete.")

0 RePORTER_PRJABS_C_FY2006.csv
1 RePORTER_PRJABS_C_FY2007.csv
2 RePORTER_PRJABS_C_FY2008.csv
3 RePORTER_PRJABS_C_FY2009.csv
4 RePORTER_PRJABS_C_FY2010.csv
5 RePORTER_PRJABS_C_FY2011.csv
6 RePORTER_PRJABS_C_FY2012.csv
7 RePORTER_PRJABS_C_FY2013.csv
8 RePORTER_PRJABS_C_FY2014.csv
9 RePORTER_PRJABS_C_FY2015.csv
10 RePORTER_PRJABS_C_FY2016.csv
11 RePORTER_PRJABS_C_FY2017.csv
12 RePORTER_PRJABS_C_FY2018.csv
13 RePORTER_PRJABS_C_FY2019.csv


In [ ]:
unique_years = sorted(set(years))
keywords = ['yeast','cancer','zebrafish','elegans','crispr']
for keyword in keywords:
    for year in unique_years:
        s,m,l = CountGrantsInOneYearWithWord( abstracts[years==year] , keyword)
        print(year,keyword,m,sep='\t')

In [ ]:
# this crashes because of file-format problems
df = pd.DataFrame()
for I in range(0,len(csv_files),2):
    print(csv_files[I])
    print(csv_files[I+1])
    df1 = pd.read_csv(csv_files[I][1],sep=',',low_memory=False)
    df2 = pd.read_csv(csv_files[I+1][1],sep=',',low_memory=False)
    df2['Year'] = csv_files[I][0]
    df1 = df1.merge(df2,on='APPLICATION_ID')
    if I==0:
        df=df1
    else:
        df.append(df1)
    
    

In [148]:
help(multiprocessing)

Help on package multiprocessing:

NAME
    multiprocessing

MODULE REFERENCE
    https://docs.python.org/3.8/library/multiprocessing
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    # Package analogous to 'threading.py' but using processes
    #
    # multiprocessing/__init__.py
    #
    # This package is intended to duplicate the functionality (and much of
    # the API) of threading.py but uses processes instead of threads.  A
    # subpackage 'multiprocessing.dummy' has the same API but is a simple
    # wrapper for 'threading'.
    #
    # Copyright (c) 2006-2008, R Oudkerk
    # Licensed to PSF under a Contributor Agreement.
    #

PACKAGE CONTENTS
    connection
    context
    dummy (packag